In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm 
from transformers import ViltProcessor, ViltModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [3]:
import os
import pandas as pd
current_dir = os.getcwd()

In [4]:

train_path = f"{current_dir}/train.csv"
test_path = f"{current_dir}/test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

cols = ['ImageFilePath', 'ScientificName']
train_df = train_df[cols].dropna()
test_df = test_df[cols].dropna()

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "dandelin/vilt-b32-mlm"

model = ViltModel.from_pretrained(model_name).eval().to(device)

processor = ViltProcessor.from_pretrained(model_name)


In [6]:
def extract_features(image_path, taxa) :
    
    image = Image.open(image_path).convert("RGB")
    
    text = f"This is an image of a beetle, of species : {taxa}"
    
    inputs = processor(images=image, text=text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    features = outputs.last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy().flatten()
    

In [7]:

X_train = np.vstack([extract_features(img, taxa) for img, taxa in tqdm(zip(train_df["ImageFilePath"], train_df["ScientificName"]), desc="Processing ....") ])

X_test = np.vstack([extract_features(img, taxa) for img, taxa in tqdm(zip(test_df["ImageFilePath"], test_df["ScientificName"]), desc="Processing ....") ])

le = LabelEncoder()

all_labels = pd.concat([train_df["ScientificName"], test_df["ScientificName"]]).unique()

le.fit(all_labels)

y_train = le.transform(train_df["ScientificName"])

y_test = le.transform(test_df["ScientificName"])

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)



Processing ....: 0it [00:00, ?it/s]

Processing ....: 0it [00:00, ?it/s]

In [8]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=11),  
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in tqdm(models.items(), desc="Training Models", leave=True):
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T



Training Models:   0%|          | 0/4 [00:00<?, ?it/s]

NaiveBayes                | Acc: 9.94% | Prec: 21.54% | Rec: 9.94% | F1: 10.31% | Bal Acc: 9.31% | MCC: 0.1185
LogisticRegression        | Acc: 47.57% | Prec: 49.98% | Rec: 47.57% | F1: 42.87% | Bal Acc: 44.96% | MCC: 0.4676


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


NearestNeighbor           | Acc: 23.57% | Prec: 33.05% | Rec: 23.57% | F1: 19.27% | Bal Acc: 19.94% | MCC: 0.2242
MLP_Baseline              | Acc: 37.89% | Prec: 47.49% | Rec: 37.89% | F1: 34.86% | Bal Acc: 35.66% | MCC: 0.3692


In [9]:

test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})

test_df.head(2)



ImageFilePath      ScientificName  \
0  /fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...   Tetracha carolina   
1  /fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...  Chlaenius aestivus   

    Pred_NaiveBayes Pred_LogisticRegression Pred_NearestNeighbor  \
0  Harpalus paratus       Tetracha carolina   Chlaenius aestivus   
1  Harpalus paratus      Chlaenius aestivus   Chlaenius aestivus   

       Pred_MLP_Baseline  
0  Cymindis cribricollis  
1     Chlaenius aestivus

In [10]:

# basedir = '/users/PAS2136/rayees/CV4A - Efficient Probing/BritishCarabids_10_per_species/'


In [11]:

test_df.to_csv(f"{current_dir}/6.ViLT.csv", index=False)

metrics_df.to_csv(f"{current_dir}/6.ViLT-metrics.csv", index=False)

